# GONet Exploration
Notebook for scratch exploration.

# Imports

In [ ]:
# stdlib
import os

In [ ]:
# External libraries
import cv2
import h5py
import numpy as np
import tqdm

In [ ]:
# Plotting libraries
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import seaborn as sns
sns.set_style('white')

# Settings

In [ ]:
rng = np.random.default_rng(1371239871)

In [ ]:
# Palos #2
image_dirs = {
    'palos': '/Users/zhafensaavedra/Library/CloudStorage/GoogleDrive-zhafensaavedra@adlerplanetarium.org/Shared drives/GONet/Version 2/Data/Field Results/230517 Palos Triangulation/230517 Synced Images',
    'conway': '/Users/zhafensaavedra/Library/CloudStorage/GoogleDrive-zhafensaavedra@adlerplanetarium.org/Shared drives/Far Horizons Program/Presentations Publications/Triangulation Experiment/Tri Exp GONet Data/221218 Conway Experiment/221218 Conway GONet Images',
}
deployment = 'conway'
image_dir = image_dirs[deployment]

In [ ]:
savedir = '/Users/zhafensaavedra/Library/CloudStorage/GoogleDrive-zhafensaavedra@adlerplanetarium.org/Shared drives/Far Horizons Program/Presentations Publications/Triangulation Experiment/Remote Sensed Data/packaged_image_data'

In [ ]:
max_angle = 90. # In degrees
dtheta = 5. # Bin size in degrees

# Explore

In [ ]:
fns = os.listdir(image_dir)

In [ ]:
fp = os.path.join(image_dir, rng.choice(fns))

In [ ]:
img = cv2.imread(fp)
img = img[:, :, ::-1]

In [ ]:
plt.imshow(
    img,
)

## Identify region where data is

In [ ]:
zero = (img < 16).astype(float)
center = (zero.shape[1]//2, zero.shape[0]//2)
radius = center[1]

In [ ]:
fig = plt.figure(figsize=(21,7))
ax_dict = fig.subplot_mosaic([['r', 'g', 'b']])

for i, band in enumerate('rgb'):
    ax = ax_dict[band]
    ax.imshow(zero[:,:,i], cmap='Greys', vmin=0, vmax=1)
    
    ax.axvline(center[0])
    ax.axhline(center[1])
    
    for r in [10, center[1]]:
        circle = patches.Circle(center, radius=r, fill=False, edgecolor='k', linewidth=2)
        ax.add_patch(circle)


## Perform Crude Resampling

### Example

In [ ]:
xs = np.arange(zero.shape[1])
ys = np.arange(zero.shape[0])
xs, ys = np.meshgrid(xs, ys)
# Center
xs = xs - center[0]
ys = ys - center[1]

In [ ]:
# Transform coords
s = np.sqrt(xs**2. + ys**2.)
# Scale to assume that the fisheye is in even steps
theta = s / radius * max_angle
phi = np.arctan2(ys, xs) * (180/np.pi)

In [ ]:
# Flatten and toss out filler
theta_flat = theta.flatten()
valid = theta_flat < max_angle
theta_flat = theta_flat[valid]
phi_flat = phi.flatten()[valid]
img_flat = [img[:,:,i].flatten()[valid] for i, band in enumerate('rgb')]

In [ ]:
# Nearest-grid-point resample (histogram bin)
theta_bins = np.arange(0., 90. + dtheta, dtheta)
phi_bins = np.arange(-180., 180. + dtheta, dtheta)
theta_centers = 0.5 * (theta_bins[:-1] + theta_bins[1:])
phi_centers = 0.5 * (phi_bins[:-1] + phi_bins[1:])

In [ ]:
mosaic = [ [f'r_{type}', f'g_{type}', f'b_{type}'] for type in ['count', 'sum', 'average']] + [ ['final', '.', '.' ], ]

In [ ]:
fig = plt.figure(figsize=(21,21))
ax_dict = fig.subplot_mosaic(mosaic)

resampled = []
for i, band in enumerate('rgb'):    

    img_float_i = img_flat[i] / 255
    
    ax = ax_dict[f'{band}_count']
    counts, _, _, _ = ax.hist2d(
        phi_flat,
        theta_flat,
        bins=(phi_bins, theta_bins),
    )
    
    ax = ax_dict[f'{band}_sum']
    sums, _, _, _ = ax.hist2d(
        phi_flat,
        theta_flat,
        weights=img_float_i,
        bins=(phi_bins, theta_bins),
    )
    
    averages = sums / counts
    resampled.append(averages)
    
    ax = ax_dict[f'{band}_average']
    ax.pcolormesh(
        phi_bins,
        theta_bins,
        averages.transpose(),
        vmin=0,
        vmax=1,
    )
resampled = np.array(resampled).transpose()
    
ax = ax_dict['final']
ax.pcolormesh(
    phi_bins,
    theta_bins,
    resampled
)
    
for ax_key, ax in ax_dict.items():
    ax.set_ylim(90, 0)

### Resample all files and package up 

In [ ]:
# Bins for resampling
theta_bins = np.arange(0., 90. + dtheta, dtheta)
phi_bins = np.arange(-180., 180. + dtheta, dtheta)
theta_centers = 0.5 * (theta_bins[:-1] + theta_bins[1:])
phi_centers = 0.5 * (phi_bins[:-1] + phi_bins[1:])

In [ ]:
imgs = {}
for fn in tqdm.tqdm(fns):
    
    fp =  os.path.join(image_dir, fn)

    # Load the image
    img = cv2.imread(fp)
    img = img[:, :, ::-1]

    # Pixel dimensions
    center = (img.shape[1]//2, img.shape[0]//2)
    radius = center[1]

    # Pixel coords
    xs = np.arange(zero.shape[1])
    ys = np.arange(zero.shape[0])
    xs, ys = np.meshgrid(xs, ys)

    # Center coords
    xs = xs - center[0]
    ys = ys - center[1]

    # Transform coords
    s = np.sqrt(xs**2. + ys**2.)
    # Scale to assume that the fisheye is in even steps
    theta = s / radius * max_angle
    phi = np.arctan2(ys, xs) * (180/np.pi)

    # Flatten and toss out filler
    theta_flat = theta.flatten()
    valid = theta_flat < max_angle
    theta_flat = theta_flat[valid]
    phi_flat = phi.flatten()[valid]
    img_flat = [img[:,:,i].flatten()[valid] for i, band in enumerate('rgb')]

    # Perform resampling
    resampled = []
    for i, band in enumerate('rgb'):    

        img_float_i = img_flat[i] / 255

        counts, _, _ = np.histogram2d(
            phi_flat,
            theta_flat,
            bins=(phi_bins, theta_bins),
        )

        ax = ax_dict[f'{band}_sum']
        sums, _, _ = np.histogram2d(
            phi_flat,
            theta_flat,
            weights=img_float_i,
            bins=(phi_bins, theta_bins),
        )

        averages = sums / counts
        resampled.append(averages)
    resampled = np.array(resampled).transpose()
    
    # Store
    key = os.path.splitext(fn)[0]
    imgs[key] = resampled

In [ ]:
# Plot the last image
plt.pcolormesh(
    phi_bins,
    theta_bins[::-1],
    resampled
)

In [ ]:
savefp = os.path.join(savedir, f'{deployment}.h5')

In [ ]:
with h5py.File(savefp, 'w') as f:
    
    # Loop through to save images
    for key, img in imgs.items():
        f.create_dataset(key, data=img, dtype=img.dtype)
        
    # Save attributes
    f.attrs.create('azimuthal_angle_bins_in_degrees', data=phi_bins)
    f.attrs.create('vertical_angle_bins_in_degrees', data=theta_bins)

In [ ]:
# Snippet for reloading the data
import h5py

f = h5py.File(savefp, 'r')

imgs = {}
for key in f.keys():
    imgs[key] = f[key][:]

azimuthal_bins = f.attrs['azimuthal_angle_bins_in_degrees'][:]
vertical_bins = f.attrs['vertical_angle_bins_in_degrees'][:]